In [ ]:
# Install required packages (runs automatically in Colab, fast no-op in Binder)
!pip install -q qiskit qiskit-aer qiskit-ibm-runtime pylatexenc

# OpenQASM 2 und das Qiskit SDK

{/*
  DO NOT EDIT THIS CELL!!!
  This cell's content is generated automatically by a script. Anything you add
  here will be removed next time the notebook is run. To add new content, create
  a new cell before or after this one.
*/}

<details>
<summary><b>Package versions</b></summary>

The code on this page was developed using the following requirements.
We recommend using these versions or newer.

```
qiskit[all]~=2.3.0
```
</details>
Das Qiskit SDK bietet einige Tools zur Konvertierung zwischen OpenQASM-Darstellungen von Quantenprogrammen und der [QuantumCircuit](https://docs.quantum.ibm.com/api/qiskit/qiskit.circuit.QuantumCircuit)-Klasse.

<span id="qasm2-import"></span>
## Importieren eines OpenQASM 2-Programms in Qiskit
Zwei Funktionen importieren OpenQASM 2-Programme in Qiskit.
Dies sind [`qasm2.load()`](../api/qiskit/qasm2#load), die einen Dateinamen akzeptiert, und [`qasm2.loads()`](../api/qiskit/qasm2#loads), die das OpenQASM 2-Programm als String akzeptiert.

In [1]:
import qiskit.qasm2

program = """
    OPENQASM 2.0;
    include "qelib1.inc";
    qreg q[2];
    creg c[2];

    h q[0];
    cx q[0], q[1];

    measure q -> c;
"""
circuit = qiskit.qasm2.loads(program)
circuit.draw()

     ┌───┐     ┌─┐   
q_0: ┤ H ├──■──┤M├───
     └───┘┌─┴─┐└╥┘┌─┐
q_1: ─────┤ X ├─╫─┤M├
          └───┘ ║ └╥┘
c: 2/═══════════╩══╩═
                0  1 

Weitere Informationen findest du in der [OpenQASM 2 Qiskit API](https://docs.quantum.ibm.com/api/qiskit/qasm2).

### Importieren einfacher Programme
Für die meisten OpenQASM 2-Programme kannst du einfach `qasm2.load` und `qasm2.loads` mit einem einzigen Argument verwenden.

#### Beispiel: Importieren eines OpenQASM 2-Programms als String
Verwende `qasm2.loads()`, um ein OpenQASM 2-Programm als String in einen QuantumCircuit zu importieren:

In [2]:
from qiskit import qasm2

program = """
    OPENQASM 2.0;
    include "qelib1.inc";

    qreg q[4];
    creg c[4];

    h q[0];
    cx q[0], q[1];

    // 'rxx' is not actually in `qelib1.inc`,
    // but Qiskit used to behave as if it were.
    rxx(0.75) q[2], q[3];

    measure q -> c;
"""
circuit = qasm2.loads(
    program,
    custom_instructions=qasm2.LEGACY_CUSTOM_INSTRUCTIONS,
)

#### Example: use a particular gate class when importing an OpenQASM 2 program

Qiskit cannot, in general, verify if the definition in an OpenQASM 2 `gate` statement corresponds exactly to a Qiskit standard-library gate.
Instead, Qiskit chooses a custom gate using the precise definition supplied.
This can be less efficient that using one of the built-in standard gates, or a user-defined custom gate.
You can manually define `gate` statements with particular classes.

In [3]:
from qiskit import qasm2
from qiskit.circuit import Gate
from qiskit.circuit.library import RZXGate


# Define a custom gate that takes one qubit and two angles.
class MyGate(Gate):
    def __init__(self, theta, phi):
        super().__init__("my", 1, [theta, phi])


custom_instructions = [
    # Link the OpenQASM 2 name 'my' with our custom gate.
    qasm2.CustomInstruction("my", 2, 1, MyGate),
    # Link the OpenQASM 2 name 'rzx' with Qiskit's
    # built-in RZXGate.
    qasm2.CustomInstruction("rzx", 1, 2, RZXGate),
]

program = """
    OPENQASM 2.0;

    gate my(theta, phi) q {
        U(theta / 2, phi, -theta / 2) q;
    }
    gate rzx(theta) a, b {
        // It doesn't matter what definition is
        // supplied, if the parameters match;
        // Qiskit will still use `RZXGate`.
    }

    qreg q[2];
    my(0.25, 0.125) q[0];
    rzx(pi) q[0], q[1];
"""

circuit = qasm2.loads(
    program,
    custom_instructions=custom_instructions,
)

#### Beispiel: Importieren eines OpenQASM 2-Programms aus einer Datei
Verwende `load()`, um ein OpenQASM 2-Programm aus einer Datei in einen QuantumCircuit zu importieren:

In [4]:
from qiskit import qasm2
from qiskit.circuit import Gate


# Define a custom gate that takes one qubit and two angles.
class MyGate(Gate):
    def __init__(self, theta, phi):
        super().__init__("my", 1, [theta, phi])


custom_instructions = [
    qasm2.CustomInstruction("my", 2, 1, MyGate, builtin=True),
]

program = """
    OPENQASM 2.0;
    qreg q[1];

    my(0.25, 0.125) q[0];
"""

circuit = qasm2.loads(
    program,
    custom_instructions=custom_instructions,
)

<span id="custom-instructions"></span>
### Verknüpfen von OpenQASM 2-Gates mit Qiskit-Gates
Standardmäßig behandelt der OpenQASM 2-Importer von Qiskit die Include-Datei `"qelib1.inc"` als *de facto* Standardbibliothek.
Der Importer behandelt diese Datei so, als enthielte sie genau die Gates, die in [dem ursprünglichen Papier, das OpenQASM 2 definiert](https://arxiv.org/abs/1707.03429), beschrieben sind.
Qiskit verwendet die integrierten Gates in [der Circuit Library](../api/qiskit/circuit_library), um die Gates in `"qelib1.inc"` darzustellen.
Gates, die im Programm durch manuelle OpenQASM 2 `gate`-Anweisungen definiert werden, werden standardmäßig als benutzerdefinierte [Qiskit `Gate`-Unterklassen](../api/qiskit/qiskit.circuit.Gate) konstruiert.

Du kannst dem Importer mitteilen, spezifische [`Gate`](../api/qiskit/qiskit.circuit.Gate)-Klassen für die von ihm gefundenen `gate`-Anweisungen zu verwenden.
Du kannst diesen Mechanismus auch verwenden, um zusätzliche Gate-Namen als "built-in" zu behandeln, d.h. keine explizite Definition zu erfordern.
Wenn du angibst, welche Gate-Klassen für `gate`-Anweisungen außerhalb von `"qelib1.inc"` verwendet werden sollen, ist die resultierende Schaltung typischerweise effizienter zu bearbeiten.

> **Warning:** Ab Qiskit SDK v1.0 verhält sich der OpenQASM 2-*Exporter* von Qiskit (siehe [Exportieren einer Qiskit-Schaltung nach OpenQASM 2](#qasm2-export)) immer noch so, als hätte `"qelib1.inc"` mehr Gates als tatsächlich vorhanden sind.
> Das bedeutet, dass die Standardeinstellungen des Importers möglicherweise kein von unserem Exporter exportiertes Programm importieren können.
> Siehe [das spezifische Beispiel zur Arbeit mit dem Legacy-Exporter](#qasm2-import-legacy), um dieses Problem zu lösen.
> 
> Dies ist Legacy-Verhalten von Qiskit, und [es wird in einer späteren Version von Qiskit behoben](https://github.com/Qiskit/qiskit/issues/10737).

Um Informationen über eine benutzerdefinierte Anweisung an den OpenQASM 2-Importer zu übergeben, verwende [die `qasm2.CustomInstruction`-Klasse](../api/qiskit/qasm2#qiskit.qasm2.CustomInstruction).
Diese hat vier erforderliche Informationen, in dieser Reihenfolge:

* Der **Name** des Gates, der im OpenQASM 2-Programm verwendet wird
* Die **Anzahl der Winkelparameter**, die das Gate akzeptiert
* Die **Anzahl der Qubits**, auf die das Gate wirkt
* Die Python-**Konstruktor**-Klasse oder -Funktion für das Gate, die die Gate-Parameter (aber nicht die Qubits) als einzelne Argumente akzeptiert

Wenn der Importer auf eine `gate`-Definition trifft, die mit einer gegebenen benutzerdefinierten Anweisung übereinstimmt, wird er diese benutzerdefinierten Informationen verwenden, um das Gate-Objekt zu rekonstruieren.
Wenn eine `gate`-Anweisung gefunden wird, die mit dem `name` einer benutzerdefinierten Anweisung übereinstimmt, aber nicht sowohl mit der Anzahl der Parameter als auch mit der Anzahl der Qubits übereinstimmt, gibt der Importer einen [`QASM2ParseError`](../api/qiskit/qasm2#qasm2parseerror) aus, um die Diskrepanz zwischen den bereitgestellten Informationen und dem Programm anzuzeigen.

Zusätzlich kann ein fünftes Argument `builtin` optional auf `True` gesetzt werden, um das Gate automatisch innerhalb des OpenQASM 2-Programms verfügbar zu machen, auch wenn es nicht explizit definiert ist.
Wenn der Importer auf eine explizite `gate`-Definition für eine integrierte benutzerdefinierte Anweisung trifft, wird er sie stillschweigend akzeptieren.
Wie zuvor wird ein [`QASM2ParseError`](../api/qiskit/qasm2#qasm2parseerror) ausgegeben, wenn eine explizite Definition desselben Namens nicht mit der bereitgestellten benutzerdefinierten Anweisung kompatibel ist.
Dies ist nützlich für die Kompatibilität mit älteren OpenQASM 2-Exportern und mit bestimmten anderen Quantenplattformen, die die "Basis-Gates" ihrer Hardware als integrierte Anweisungen behandeln.

Qiskit bietet ein Datenattribut für die Arbeit mit OpenQASM 2-Programmen, die von Legacy-Versionen von [Qiskits OpenQASM 2-Exportfunktionen](#qasm2-export) erstellt wurden.
Dies ist [`qasm2.LEGACY_CUSTOM_INSTRUCTIONS`](../api/qiskit/qasm2#legacy-compatibility), das als `custom_instructions`-Argument an [`qasm2.load()`](../api/qiskit/qasm2#load) und [`qasm2.loads()`](../api/qiskit/qasm2#loads) übergeben werden kann.

<span id="qasm2-import-legacy"></span>
#### Beispiel: Importieren eines Programms, das vom Legacy-Exporter von Qiskit erstellt wurde
Dieses OpenQASM 2-Programm verwendet Gates, die nicht in der ursprünglichen Version von `"qelib1.inc"` enthalten sind, ohne sie zu deklarieren, aber Standard-Gates in Qiskits Bibliothek sind.
Du kannst [`qasm2.LEGACY_CUSTOM_INSTRUCTIONS`](../api/qiskit/qasm2#legacy-compatibility) verwenden, um dem Importer einfach mitzuteilen, dass er denselben Satz von Gates verwenden soll, den Qiskits OpenQASM 2-Exporter zuvor verwendet hat.

In [5]:
import math
from qiskit import qasm2

program = """
    include "qelib1.inc";
    qreg q[2];
    rx(arctan(pi, 3 + add_one(0.2))) q[0];
    cx q[0], q[1];
"""


def add_one(x):
    return x + 1


customs = [
    # Our `add_one` takes only one parameter.
    qasm2.CustomClassical("add_one", 1, add_one),
    # `arctan` takes two parameters, and `math.atan2` implements it.
    qasm2.CustomClassical("arctan", 2, math.atan2),
]
circuit = qasm2.loads(program, custom_classical=customs)

#### Beispiel: Verwendung einer bestimmten Gate-Klasse beim Importieren eines OpenQASM 2-Programms
Qiskit kann im Allgemeinen nicht überprüfen, ob die Definition in einer OpenQASM 2 `gate`-Anweisung genau einem Gate der Qiskit-Standardbibliothek entspricht.
Stattdessen wählt Qiskit ein benutzerdefiniertes Gate unter Verwendung der bereitgestellten genauen Definition.
Dies kann weniger effizient sein als die Verwendung eines der integrierten Standard-Gates oder eines benutzerdefinierten Gates.
Du kannst manuell `gate`-Anweisungen mit bestimmten Klassen definieren.

In [6]:
from qiskit import QuantumCircuit, qasm2

# Define any circuit.
circuit = QuantumCircuit(2, 2)
circuit.h(0)
circuit.cx(0, 1)
circuit.measure([0, 1], [0, 1])

# Export to a string.
program = qasm2.dumps(circuit)

# Export to a file.
qasm2.dump(circuit, "my_file.qasm")

#### Beispiel: Definieren eines neuen integrierten Gates in einem OpenQASM 2-Programm
Wenn das Argument `builtin=True` gesetzt ist, benötigt ein benutzerdefiniertes Gate keine zugehörige Definition.